In [19]:
#preprocessing of pattern for Boyer More algorithm
import string

def z_array(s):
    """ Use Z algorithm (Gusfield theorem 1.4.1) to preprocess s """
    assert len(s) > 1
    z = [len(s)] + [0] * (len(s)-1)
    # Initial comparison of s[1:] with prefix
    for i in range(1, len(s)):
        if s[i] == s[i-1]:
            z[1] += 1
        else:
            break
    r, l = 0, 0
    if z[1] > 0:
        r, l = z[1], 1
    for k in range(2, len(s)):
        assert z[k] == 0
        if k > r:
            # Case 1
            for i in range(k, len(s)):
                if s[i] == s[i-k]:
                    z[k] += 1
                else:
                    break
            r, l = k + z[k] - 1, k
        else:
            # Case 2
            # Calculate length of beta
            nbeta = r - k + 1
            zkp = z[k - l]
            if nbeta > zkp:
                # Case 2a: Zkp wins
                z[k] = zkp
            else:
                # Case 2b: Compare characters just past r
                nmatch = 0
                for i in range(r+1, len(s)):
                    if s[i] == s[i - k]:
                        nmatch += 1
                    else:
                        break
                l, r = k, r + nmatch
                z[k] = r - k + 1
    return z


def n_array(s):
    """ Compile the N array (Gusfield theorem 2.2.2) from the Z array """
    return z_array(s[::-1])[::-1]


def big_l_prime_array(p, n):
    """ Compile L' array (Gusfield theorem 2.2.2) using p and N array.
        L'[i] = largest index j less than n such that N[j] = |P[i:]| """
    lp = [0] * len(p)
    for j in range(len(p)-1):
        i = len(p) - n[j]
        if i < len(p):
            lp[i] = j + 1
    return lp


def big_l_array(p, lp):
    """ Compile L array (Gusfield theorem 2.2.2) using p and L' array.
        L[i] = largest index j less than n such that N[j] >= |P[i:]| """
    l = [0] * len(p)
    l[1] = lp[1]
    for i in range(2, len(p)):
        l[i] = max(l[i-1], lp[i])
    return l


def small_l_prime_array(n):
    """ Compile lp' array (Gusfield theorem 2.2.4) using N array. """
    small_lp = [0] * len(n)
    for i in range(len(n)):
        if n[i] == i+1:  # prefix matching a suffix
            small_lp[len(n)-i-1] = i+1
    for i in range(len(n)-2, -1, -1):  # "smear" them out to the left
        if small_lp[i] == 0:
            small_lp[i] = small_lp[i+1]
    return small_lp


def good_suffix_table(p):
    """ Return tables needed to apply good suffix rule. """
    n = n_array(p)
    lp = big_l_prime_array(p, n)
    return lp, big_l_array(p, lp), small_l_prime_array(n)


def good_suffix_mismatch(i, big_l_prime, small_l_prime):
    """ Given a mismatch at offset i, and given L/L' and l' arrays,
        return amount to shift as determined by good suffix rule. """
    length = len(big_l_prime)
    assert i < length
    if i == length - 1:
        return 0
    i += 1  # i points to leftmost matching position of P
    if big_l_prime[i] > 0:
        return length - big_l_prime[i]
    return length - small_l_prime[i]


def good_suffix_match(small_l_prime):
    """ Given a full match of P to T, return amount to shift as
        determined by good suffix rule. """
    return len(small_l_prime) - small_l_prime[1]


def dense_bad_char_tab(p, amap):
    """ Given pattern string and list with ordered alphabet characters, create
        and return a dense bad character table.  Table is indexed by offset
        then by character. """
    tab = []
    nxt = [0] * len(amap)
    for i in range(0, len(p)):
        c = p[i]
        assert c in amap
        tab.append(nxt[:])
        nxt[amap[c]] = i+1
    return tab


class BoyerMoore(object):
    """ Encapsulates pattern and associated Boyer-Moore preprocessing. """
    
    def __init__(self, p, alphabet='ACGT'):
        self.p = p
        self.alphabet = alphabet
        # Create map from alphabet characters to integers
        self.amap = {}
        for i in range(len(self.alphabet)):
            self.amap[self.alphabet[i]] = i
        # Make bad character rule table
        self.bad_char = dense_bad_char_tab(p, self.amap)
        # Create good suffix rule table
        _, self.big_l, self.small_l_prime = good_suffix_table(p)
    
    def bad_character_rule(self, i, c):
        """ Return # skips given by bad character rule at offset i """
        assert c in self.amap
        ci = self.amap[c]
        assert i > (self.bad_char[i][ci]-1)
        return i - (self.bad_char[i][ci]-1)
    
    def good_suffix_rule(self, i):
        """ Given a mismatch at offset i, return amount to shift
            as determined by (weak) good suffix rule. """
        length = len(self.big_l)
        assert i < length
        if i == length - 1:
            return 0
        i += 1  # i points to leftmost matching position of P
        if self.big_l[i] > 0:
            return length - self.big_l[i]
        return length - self.small_l_prime[i]
    
    def match_skip(self):
        """ Return amount to shift in case where P matches T """
        return len(self.small_l_prime) - self.small_l_prime[1]

In [8]:
def naive(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):
        match = True
        for j in range(len(p)):
            if t[i+j] != p[j]:
                match = False
                break
        if match:
            occurrences.append(i)
    return occurrences

def naive_with_counts(p,t):
    """Naive exact matching function modified to count the number of character comparisions and 
    number of alignments checked for to get the matches"""
    occurrences = []
    nc=0 # no. of character comparisions
    na=0 # no. of alignments made
    for i in range(len(t) - len(p) + 1):
        mm=0
        match = True
        na+=1
        for j in range(len(p)):
            nc+=1
            if t[i+j] != p[j]:
                match = False
                break
        if match:
            occurrences.append(i)
    return occurrences,na,nc

def boyer_moore_with_counts(p, p_bm, t):
    """ Boyer moore module modified to count the number of character comparisions and 
    number of alignments checked for to get the matches"""
    i = 0
    occurrences = []
    nc=0 # no. of character comparisions
    na=0 # no. of alignments made
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        na+=1
        for j in range(len(p)-1, -1, -1):
            nc+=1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences, na, nc



In [15]:
"""Testing the Naive exacting matching module which additionally returns the number of character comparisions done
and number of alignment matches checked for along with the occurance of p in t"""

p = 'word'
t = 'there would have been a time for such a word'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print("The pattern {",p,"} occurs in the the text {",t,"} at ",occurrences,"position/s")
print("The number of character comparisions done by the naive exact matching algorithm is", num_character_comparisons, "and the number of aligments checked for:",num_alignments)

p = 'needle'
t = 'needle need noodle needle'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print("The pattern {",p,"} occurs in the the text {",t,"} at ",occurrences,"position/s")
print("The number of character comparisions done by the naive exact matching algorithm is", num_character_comparisons, "and the number of aligments checked for:",num_alignments)

The pattern { word } occurs in the the text { there would have been a time for such a word } at  [40] position/s
The number of character comparisions done by the naive exact matching algorithm is 46 and the number of aligments checked for: 41
The pattern { needle } occurs in the the text { needle need noodle needle } at  [0, 19] position/s
The number of character comparisions done by the naive exact matching algorithm is 35 and the number of aligments checked for: 20


In [23]:
"""Testing the boyer_moore_with_counts module which additionally returns the number of character comparisions done
and number of alignment matches checked for along with the occurance of p in t"""

p = 'word'
t = 'there would have been a time for such a word'
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print("The pattern {",p,"} occurs in the the text {",t,"} at ",occurrences,"position/s")
print("The number of character comparisions done by the naive exact matching algorithm is", num_character_comparisons, "and the number of aligments checked for:",num_alignments)


p = 'needle'
t = 'needle need noodle needle'
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print("The pattern {",p,"} occurs in the the text {",t,"} at ",occurrences,"position/s")
print("The number of character comparisions done by the naive exact matching algorithm is", num_character_comparisons, "and the number of aligments checked for:",num_alignments)

The pattern { word } occurs in the the text { there would have been a time for such a word } at  [40] position/s
The number of character comparisions done by the naive exact matching algorithm is 15 and the number of aligments checked for: 12
The pattern { needle } occurs in the the text { needle need noodle needle } at  [0, 19] position/s
The number of character comparisions done by the naive exact matching algorithm is 18 and the number of aligments checked for: 5


In [26]:
"""Index assisted approximate matching using the peigion hole principle to allow for mismatches"""
#To create kmer indices that can be strored and checked for
import bisect

class Index(object):
    """ Holds a substring index for a text T """

    def __init__(self, t, k):
        """ Create index from all substrings of t of length k """
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer

    def query(self, p):
        """ Return index hits for first k-mer of p """
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits


def queryIndex(p, t, index):
    """checks for matchs of pattern p in text t (index created from t using the bisect class))"""
    k = index.k
    offsets = []
    for i in index.query(p):
        if p[k:] == t[i+k:i+len(p)]:  # verify that rest of P matches (if the query in index object matches)
            offsets.append(i)
    return offsets

# arguments- pattern p, text t and number of mismatches allowed (Peigion hole principle)
def approximate_match(p, t, n):
    """Returns the occurances of pattern p in text t allowing upto n mismatches and also the number of index hits during the process""" 
    segment_length = int(round(len(p) / (n+1))) #length- 1+ number of mismatches allowed interger conversion
    all_matches = set() # set of all the mismatches
    nm=0 # no. of index hits
    for i in range(n+1):
        start = i*segment_length
        end = min((i+1)*segment_length, len(p)) # p's length will not be a perfect multiple of n and so should not run out of it
        matches = queryIndex(p[start:end], t, index) 
        nm+=len(matches)
        # Extend matching segments to see if whole p matches (Verification)
        for m in matches:
            if m < start or m-start+len(p) > len(t):  #the matched position should not run out from the start ot end of the length of t
                continue # so skip rest
            mismatches = 0
            for j in range(0, start): # befor the matched position
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)): # after the matched position
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(m - start)
    return list(all_matches), nm # set will remove duplicates

def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome
hs_genome = readGenome('chr1.GRCh38.excerpt (1).fasta')

In [30]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
t=hs_genome
index = Index(hs_genome, 8)
occurance, no_matches=approximate_match(p, t, 2)
print("The pattern{",p,"occurns",len(occurance),"times in hs_genome(chr1 excerpt) and the number of index hits using the Index assisted approximate matching alogorithm allowing 2 mismatches are ",no_matches)

The pattern{ GGCGCGGTGGCTCACGCCTGTAAT occurns 19 times in hs_genome(chr1 excerpt) and the number of index hits using the Index assisted approximate matching alogorithm allowing 2 mismatches are  90
